In [1]:
!pip install transformers
!pip install sentencepice
!git clone "https://github.com/aarushi211/TOS-Summarization.git"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 11.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement sentencepice (from versions: none)
ERROR: No matching distribution found for sentencepice
Cloning into 'TOS-Summarization'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 92 (delta 1), reused 0 (delta 0), pack-reused 82
Unpacking objects: 100% (92/92), 648.59 KiB | 4.95 MiB/s, done.


In [2]:
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
from transformers import BartTokenizer, BartForConditionalGeneration, PretrainedConfig
import torch
import pandas as pd
from rouge import Rouge

In [24]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/all_v1_transpose.csv")
df = df[['original_text','reference_summary']]
df.rename(columns = {'original_text':'source', 'reference_summary':'target'}, inplace = True)

In [25]:
df.head()

,source,target
0,welcome to the pokémon go video game services ...,hi.
1,by using our services you are agreeing to thes...,by playing this game you agree to these terms....
2,if you want to use certain features of the ser...,you have to use google pokemon trainer club or...
3,during game play please be aware of your surro...,don t die or hurt others and if you do it s no...
4,subject to your compliance with these terms ni...,don t copy modify resell distribute or reverse...


In [26]:
X = df['source']
y = df['target']

In [27]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [29]:
config = PretrainedConfig.from_json_file('/content/drive/MyDrive/Colab Notebooks/ouput_models/config.json')

In [30]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base', config = config)
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base', config = config).to(device)

In [31]:
def generate_summary(text):
    input_ids = tokenizer.encode(text, padding='longest', return_tensors='pt')
    summary_ids = model.generate(input_ids, num_beams=9,
                                  no_repeat_ngram_size=3,
                                  length_penalty=2.0,
                                  min_length=50,
                                  max_length=150,
                                  early_stopping=True)
    summary = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
    return summary

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)   

In [33]:
y_pred = X_test.apply(lambda x: generate_summary(x))

In [34]:
y_pred.iloc[0]

'in addition when you engage with advertisements that are displayed on the services novelty third party advertisers and data service providers may use industry wide tracking methods e g cookies web beacons etc. which store certain information methods and which allow us or such third parties to collect certain data for the purpose of serving advertisements through the use of the novelty services including without limitation serving advertisements that may be of interest to you.'

In [35]:
y_test.iloc[0]

'this service allows tracking via third party cookies for purposes including targeted advertising.'

In [36]:
generate_summary

<function __main__.generate_summary(text)>

In [37]:
summary = pd.concat([y_test.to_frame(name="reference_summary"), y_pred.to_frame(name="generated_summary")], axis=1)

In [38]:
summary.head()

,reference_summary,generated_summary
438,this service allows tracking via third party c...,in addition when you engage with advertisement...
415,the service may keep a secure anonymized recor...,after this time the data will either be delete...
301,you are responsible for maintaining the securi...,if you have created a user account you bear th...
409,this service is only available to users of a c...,if you are aged 18 or under and would like to ...
141,this service can share your personal informati...,by voluntarily providing us with personal info...


In [39]:
rouge = Rouge()

In [40]:
rouge.get_scores(summary['generated_summary'], summary['reference_summary'],avg=True)

{'rouge-1': {'r': 0.4853413879070315,
  'p': 0.14807778537460983,
  'f': 0.2155951124615731},
 'rouge-2': {'r': 0.16377629717682277,
  'p': 0.039979639304654715,
  'f': 0.05991382616201859},
 'rouge-l': {'r': 0.4277827440391378,
  'p': 0.1290192426574782,
  'f': 0.1879969570755829}}